In [15]:
from os.path import join
from feature.catfeature_dataset import make_dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

train_path = join('jeju_data', 'train_new.parquet')
test_path = join('jeju_data', 'test_new.parquet')
holiday_path = join('jeju_data', '국가공휴일.csv')

x_train, y_train, test = make_dataset(train_path, test_path, holiday_path)

Start time:  2022-10-31 18:59:43.583847
Train dataset success !
Test dataset success !
End time: 2022-10-31 19:01:52.690625
Play time:  0:02:09.106778


In [16]:
from category_encoders import *

In [17]:
import numpy as np
str_col = ['day_of_week', 'start_turn_restricted', 'end_turn_restricted', 'group_time', 'season', 'vacation', 'turn_restricted', 'weight_restricted']

te = TargetEncoder(cols=str_col)
train_df = te.fit_transform(x_train, y_train)
test_df = te.transform(test)

c:\Python\Python38\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
c:\Python\Python38\lib\site-packages\category_encoders\target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [18]:
X = train_df.copy()
y = y_train.copy()
test = test_df.copy()

In [23]:
X

,day_of_week,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,height_restricted,road_type,start_latitude,...,week,post_holiday,pre_holiday,holiday,season,vacation,distance,turn_restricted,rest,location_cluster
0,42.768345,1,106,0,0,60.0,47.453350,0.0,3,33.427747,...,0.0,0.0,0.0,0.0,40.340025,43.208949,0.025711,44.210368,0.0,2
1,42.768345,2,103,0,0,60.0,40.955696,0.0,0,33.500730,...,0.0,0.0,0.0,0.0,40.340025,41.792212,0.525891,34.532295,0.0,1
2,43.179300,2,103,0,0,80.0,40.955696,0.0,0,33.279145,...,1.0,1.0,1.0,0.0,43.610795,43.208949,0.608399,44.210368,1.0,3
3,42.450328,2,107,0,0,50.0,40.955696,0.0,0,33.246081,...,0.0,0.0,0.0,0.0,43.113682,43.208949,0.107352,44.210368,0.0,0
4,42.699198,2,103,0,0,80.0,40.955696,0.0,0,33.462214,...,0.0,0.0,1.0,0.0,43.610795,43.208949,0.337949,44.210368,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,42.768345,1,107,0,0,50.0,40.955696,0.0,0,33.422145,...,0.0,0.0,0.0,0.0,43.610795,43.208949,0.427004,44.210368,0.0,3
4701213,42.768345,2,107,0,0,80.0,57.883888,0.0,3,33.472505,...,0.0,0.0,0.0,0.0,43.113682,43.208949,0.048464,44.210368,0.0,1
4701214,42.761365,2,103,0,0,60.0,40.955696,0.0,0,33.447183,...,0.0,0.0,0.0,0.0,40.340025,43.208949,0.342185,44.210368,0.0,2
4701215,42.770372,2,103,0,0,80.0,40.955696,0.0,0,33.443596,...,0.0,0.0,0.0,0.0,43.610795,43.208949,0.209716,44.210368,0.0,1


In [19]:
params = {'n_estimators': 4519, 'max_depth': 13, 'min_child_weight': 22, 'gamma': 3, 'learning_rate': 0.016, 'colsample_bytree': 0.7242144738048932, 'lambda': 0.007923992135983554, 'alpha': 0.13203244419302776, 'subsample': 0.8}

xgb_model = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0).fit(X, y)
y_pred = xgb_model.predict(test)

import pandas as pd

sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
sample_submission['target'] = y_pred
sample_submission.to_csv("./submit_xgb_optunadelout.csv", index=False)

from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


params = {'n_estimators': 4519, 'max_depth': 13, 'min_child_weight': 22, 'gamma': 3, 'learning_rate': 0.016, 'colsample_bytree': 0.7242144738048932, 'lambda': 0.007923992135983554, 'alpha': 0.13203244419302776, 'subsample': 0.8}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

XGB_model= {}

for f in range(5):
      print(f'===================================={f+1}============================================')
      train_idx, val_idx = folds[f]
      
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]
      
      XGB = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0)
      XGB.fit(x_train, y_train)
      
      y_pred = XGB.predict(x_val)
      mae = mean_absolute_error(y_val, y_pred)
      print(f"{f + 1} Fold MAE = {mae}")
      XGB_model[f] = XGB
      print(f'================================================================================\n\n')
              

c:\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


====================================1============================================
1 Fold MAE = 3.0224966260874364


====================================2============================================
2 Fold MAE = 3.0211470409492485


====================================3============================================
3 Fold MAE = 3.0186674187382567


====================================4============================================
4 Fold MAE = 3.019047051892736


====================================5============================================
5 Fold MAE = 3.0180018259067003




In [20]:
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
for fold in range(5):
    sample_submission['target'] += XGB_model[fold].predict(test) / 5
    
sample_submission.to_csv("./submit_xgb_fold.csv", index=False)

In [24]:
df_imp = pd.DataFrame({'importances':XGB.feature_importances_}, index = XGB.feature_names_in_)
df_imp = df_imp[df_imp.importances > 0].sort_values('importances').copy()
df_imp

,importances
post_holiday,0.000798
week,0.000826
day_of_week,0.000894
pre_holiday,0.000952
rest,0.001008
holiday,0.001054
vacation,0.001132
multi_linked,0.001543
month,0.001693
sin_time,0.003224
